In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#MongoDB Module
from MongoCRUD import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "pickle123"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


In [4]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
# image_filename = 'my-image.png' # replace with your own image
# encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    #Use div with display flex to align items horizontally and center
    html.Div([
        html.Img(src='assets/Grazioso Salvare Logo.png', style={'height':'100px', 'width':'100px'}), 
        html.H1('SNHU CS-340 Dashboard', style={'marginLeft': '20px'}),
    ], style={'display': 'flex', 'alignItems':'center', 'justifyContent':'center'}),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),

        # Enabling single row selection
        row_selectable='single',

        # Enabling sorting
        sort_action='native',

        # Enabling filtering
        filter_action='native',

        # Enabling pagination
        page_action='native',
        page_current=0,  # page number that the user is currently on
        page_size=15,  # number of rows visible per page

        # Conditional formatting can be added here based on your requirements

        # Additional optional features for enhancing user interaction
        style_table={'overflowX': 'auto'},  # Horizontal scroll
        style_cell={
            'minWidth': '180px', 'width': '180px', 'maxWidth': '180px',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis',
        },  # Ensuring text in each cell is visible without altering the table layout
        style_header={
            'backgroundColor': 'white',
            'fontWeight': 'bold'
        },  # Styling for the header to differentiate from row entries
),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])


app.run_server(debug=True)

Dash app running on http://127.0.0.1:25127/


In [5]:
#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
## FIX ME Add code to filter interactive data table with MongoDB queries
#
#        
#        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
#        data=df.to_dict('records')
#       
#       
#        return (data,columns)

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #

    #return [
    #    dcc.Graph(            
    #        figure = px.pie(df, names='breed', title='Preferred Animals')
    #    )    
    #]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
# def update_map(viewData, index):  
#     if viewData is None:
#         return
#     elif index is None:
#         return
    
#     dff = pd.DataFrame.from_dict(viewData)
#     # Because we only allow single row selection, the list can be converted to a row index here
#     if index is None:
#         row = 0
#     else: 
#         row = index[0]
        
#     # Austin TX is at [30.75,-97.48]
#     return [
#         dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
#             dl.TileLayer(id="base-layer-id"),
#             # Marker with tool tip and popup
#             # Column 13 and 14 define the grid-coordinates for the map
#             # Column 4 defines the breed for the animal
#             # Column 9 defines the name of the animal
#             dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
#                 dl.Tooltip(dff.iloc[row,4]),
#                 dl.Popup([
#                     html.H1("Animal Name"),
#                     html.P(dff.iloc[row,9])
#                 ])
#             ])
#         ])
#     ]

def update_map(viewData, selected_rows):
    #default center and zoom
    center = [20,0]
    zoom = 2
    
    if viewData is None or not selected_rows:
        #Return default if no row is selected
        return [dl.Map(center=center, zoom=zoom, children=[dl.TileLayer()], style={'width':'1000px', 'height':'500px'})]        


    # This conversion step is necessary to access the selected row correctly
    rows = pd.DataFrame.from_records(viewData)  # Convert viewData to a DataFrame
    
    # Now, you can access the selected_row as shown
    if selected_rows and viewData and selected_rows[0] < len(rows):
        
        selected_row = rows.iloc[selected_rows[0]] #if row is selected row
        latitude = selected_row['location_lat']
        longitude = selected_row['location_long']
        center = [latitude, longitude] #center on selected row
        zoom = 10 #Zoom in to location
        
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
        marker = dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        
        #Updating the map's children to include the new marker
        return [dl.Map(center=center, zoom=zoom, children=[
            dl.TileLayer(),
            marker
        ], style={'width': '1000px', 'height': '500px'})]

app.run_server(debug=True)

IndentationError: expected an indented block (3401500539.py, line 22)